### Changes to Opioid Addiction Treatment due to COVID-19

The question has been asked, has there been any changes to opioid addiction treatment (OAT) prescribng due to the COVID-19 pandemic, such as
- were patients on daily dispensed medicines were generally switched to weekly or fortnightly dispensing
- were patients on methadone were switched to buprenorphine?

Are we able to discern any information from this, either using the OpenPrescribing dataset or via OpenSAFELY?  By undertaking a brief review of the data held in OpenPrescribing we should be able to understand the limitations of both datasets, as:
- OpenPrescribing holds prescribing from both primary care and other community commissioned services, such as Local Authority-commissioned OAT services
- OpenSAFELY only holds data from primary care
- What is the structure of OpenPrescribing data in relation to installment prescribing on prescriptions such as FP10(MDA)?

In [68]:
import pandas as pd
import os
from ebmdatalab import bq

In [69]:
#get prescribing data from BigQuery
sql ='''
#create subquery to produce single table with all different commissioned sites and organisations
WITH org_types AS
(SELECT site_code, site_name, org_code, org_name, org.region AS region, org.stp AS stp, org.org_type  FROM `ebmdatalab.richard.oat_orgs_all` AS org
INNER JOIN
`ebmdatalab.richard.oat_sites_all` AS site
ON
org.org_code = site.parent)

SELECT rx.pct, org_code, org_name, org_type, bnf.chemical AS chem_sub, RTRIM(bnf_name) as bnf_name, quantity_per_item,
SUM(CASE WHEN month = '2020-01-01' THEN items ELSE 0 END) AS jan_2020_items, # calculate January 2020 items
SUM(CASE WHEN month = '2021-01-01' THEN items ELSE 0 END) AS jan_2021_items # calculate January 2021 items
FROM ebmdatalab.hscic.raw_prescribing_normalised AS rx
INNER JOIN
org_types AS org
ON
rx.practice = org.site_code
INNER JOIN
hscic.bnf AS bnf
ON
rx.bnf_code = bnf.presentation_code
WHERE bnf_code LIKE '0410030%'
AND month IN ('2021-01-01','2020-01-01')
GROUP BY pct, org_code, org_name, org_type, chem_sub,  bnf_name, quantity_per_item
ORDER BY quantity_per_item
'''
exportfile = os.path.join("..","data","oat_df.csv") #set path for data cache
oat_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True) #save dataframe to csv
display(oat_df) #show dataframe as a table

,pct,org_code,org_name,org_type,chem_sub,bnf_name,quantity_per_item,jan_2020_items,jan_2021_items
0,72Q,72Q,NHS SOUTH EAST LONDON CCG,CCG,Buprenorphine hydrochloride,Buprenorphine 2mg sublingual tablets sugar free,1.0,1,1
1,107,107,NEWCASTLE CITY COUNCIL,LOCAL AUTH,Buprenorphine hydrochloride,Buprenorphine 8mg sublingual tablets sugar free,1.0,11,12
2,36L,36L,NHS SOUTH WEST LONDON CCG,CCG,Buprenorphine hydrochloride,Buprenorphine 2mg sublingual tablets sugar free,1.0,16,3
3,216,216,NORTH EAST LINCOLNSHIRE COUNCIL,LOCAL AUTH,Buprenorphine hydrochloride,Espranor 8mg oral lyophilisates,1.0,4,1
4,DMH,DMH,FRAMEWORK,INDEP SECTOR,Buprenorphine hydrochloride,Subutex 8mg sublingual tablets,1.0,0,1
...,...,...,...,...,...,...,...,...,...
34164,NMS,NMS,"CHANGE, GROW, LIVE",INDEP SECTOR,Methadone hydrochloride,Methadone 1mg/ml oral solution sugar free,3080.0,2,0
34165,NMS,NMS,"CHANGE, GROW, LIVE",INDEP SECTOR,Methadone hydrochloride,Physeptone 1mg/ml oral solution sugar free,3120.0,0,1
34166,01K,01K,NHS MORECAMBE BAY CCG,CCG,Methadone hydrochloride,Methadone 1mg/ml oral solution,3500.0,1,1
34167,11J,11J,NHS DORSET CCG,CCG,Methadone hydrochloride,Methadone 1mg/ml oral solution sugar free,4200.0,0,1


In [70]:
#create total volume quantities in df
oat_df["jan_2020_tot_quantity"] = oat_df["quantity_per_item"] * oat_df["jan_2020_items"]
oat_df["jan_2021_tot_quantity"] = oat_df["quantity_per_item"] * oat_df["jan_2021_items"]

### What were total quantity differences between January 2020 and Jan 2021

In [71]:
#create change volume changes between Jan 2020 and Jan 2021
# create table to show which type of organisation prescribe OAT
oat_tot_df = oat_df.groupby(['chem_sub'])[["jan_2020_tot_quantity", "jan_2021_tot_quantity"]].sum()
display(oat_tot_df)

,jan_2020_tot_quantity,jan_2021_tot_quantity
chem_sub,,
Buprenorph hydrochloride/naloxone hydrochloride,15745.0,13141.0
Buprenorphine hydrochloride,1231323.0,1344562.0
Methadone hydrochloride,103197482.0,104382686.0
Naltrexone hydrochloride,36408.0,35173.0


There appears to be a small increase in buprenorphine prescribing, although there is little change in methadone prescribing.

### Which organisations prescribed OAT?

#### Absolute numbers

In [72]:
# create table to show which type of organisation prescribe OAT
oat_org_df = oat_df.groupby(['chem_sub','org_type'])[["jan_2020_tot_quantity", "jan_2021_tot_quantity"]].sum()
oat_org_df.unstack(0)

jan_2020_tot_quantity  \
chem_sub     Buprenorph hydrochloride/naloxone hydrochloride   
org_type                                                       
CCG                                                   7922.0   
INDEP SECTOR                                          3492.0   
LOCAL AUTH                                            2771.0   
TRUST                                                 1560.0   

                                                                  \
chem_sub     Buprenorphine hydrochloride Methadone hydrochloride   
org_type                                                           
CCG                             419579.0              31923610.0   
INDEP SECTOR                    577841.0              50525851.0   
LOCAL AUTH                      167033.0              13487347.0   
TRUST                            66870.0               7260674.0   

                                       \
chem_sub     Naltrexone hydrochloride   
org_type                                
CCG                           28238.0   
INDEP SECTOR                   6328.0   
LOCAL AUTH                     1492.0   
TRUST                           350.0   

                                       jan_2021_tot_quantity  \
chem_sub     Buprenorph hydrochloride/naloxone hydrochloride   
org_type                                                       
CCG                                                   5941.0   
INDEP SECTOR                                          2838.0   
LOCAL AUTH                                            3256.0   
TRUST                                                 1106.0   

                                                                  \
chem_sub     Buprenorphine hydrochloride Methadone hydrochloride   
org_type                                                           
CCG                             399714.0              28787152.0   
INDEP SECTOR                    708989.0              55260625.0   
LOCAL AUTH                      168438.0              13162325.0   
TRUST                            67421.0               7172584.0   

                                       
chem_sub     Naltrexone hydrochloride  
org_type                               
CCG                           26962.0  
INDEP SECTOR                   6365.0  
LOCAL AUTH                     1510.0  
TRUST                           336.0

#### Percentages

In [73]:
percents_df = oat_org_df.groupby(level=0).apply(lambda x: 100 * x / (x.sum()))
percents_df.unstack(0)

jan_2020_tot_quantity  \
chem_sub     Buprenorph hydrochloride/naloxone hydrochloride   
org_type                                                       
CCG                                                50.314386   
INDEP SECTOR                                       22.178469   
LOCAL AUTH                                         17.599238   
TRUST                                               9.907907   

                                                                  \
chem_sub     Buprenorphine hydrochloride Methadone hydrochloride   
org_type                                                           
CCG                            34.075462               30.934485   
INDEP SECTOR                   46.928466               48.960353   
LOCAL AUTH                     13.565328               13.069454   
TRUST                           5.430744                7.035708   

                                       \
chem_sub     Naltrexone hydrochloride   
org_type                                
CCG                         77.559877   
INDEP SECTOR                17.380795   
LOCAL AUTH                   4.098000   
TRUST                        0.961327   

                                       jan_2021_tot_quantity  \
chem_sub     Buprenorph hydrochloride/naloxone hydrochloride   
org_type                                                       
CCG                                                45.209649   
INDEP SECTOR                                       21.596530   
LOCAL AUTH                                         24.777414   
TRUST                                               8.416407   

                                                                  \
chem_sub     Buprenorphine hydrochloride Methadone hydrochloride   
org_type                                                           
CCG                            29.728194               27.578474   
INDEP SECTOR                   52.730108               52.940413   
LOCAL AUTH                     12.527351               12.609682   
TRUST                           5.014347                6.871431   

                                       
chem_sub     Naltrexone hydrochloride  
org_type                               
CCG                         76.655389  
INDEP SECTOR                18.096267  
LOCAL AUTH                   4.293066  
TRUST                        0.955278

As you can see from above, only about 25-30% of OAT prescribing takes place on a CCG-commissioned provider (i.e. primary care).  This means that the majority of prescribing is unlikely to have a record in OpenSAFELY.

### What quantities are prescribed?

Using methadone 1mg/ml oral solution as an example, we can see what the data considers to be a "prescription", i.e. whether it is the daily installment dispensed to the patient, or whether it is the total quantity on a single FP10(MDA), which is usually for 14 days.

In [74]:
#create list of quantities of prescribing of methadone and physeptone liquid
meth_list =['Physeptone 1mg/ml oral solution sugar free', 'Methadone 1mg/ml oral solution sugar free','Methadone 1mg/ml oral solution']
meth_df = oat_df[oat_df['bnf_name'].isin(meth_list)].groupby(['quantity_per_item']).sum()
meth_df = meth_df.sort_values('jan_2021_items', ascending=False)
meth_df.head(20) # select top 20 quantities of items prescribed in January 2021


,jan_2020_items,jan_2021_items,jan_2020_tot_quantity,jan_2021_tot_quantity
quantity_per_item,,,,
840.0,13256,15855,11135040.0,13318200.0
700.0,9239,10912,6467300.0,7638400.0
980.0,8267,9870,8101660.0,9672600.0
560.0,8022,8918,4492320.0,4994080.0
420.0,6857,7507,2879940.0,3152940.0
1120.0,6487,7186,7265440.0,8048320.0
910.0,4127,4027,3755570.0,3664570.0
630.0,3237,3540,2039310.0,2230200.0
280.0,3443,3472,964040.0,972160.0


As per above, the 20 most common quantities prescribed in January 2021 are all quantities which appear to be divisible by 14, and are too high to be quantities for a single day's supply of methadone.  Consquently, it appears that OpenPrescribing data only holds the data for the entire prescription, not for daily installments.